In [ ]:
!pip install torch
!pip install tensorflow
!pip install fer
!pip install easyocr
!pip install transformers

In [ ]:
import os

new_directory = "/data/notebook_files/Reddit_Project"
os.chdir(new_directory)

In [ ]:
import torch
from fer import FER
import pandas as pd
from transformers import pipeline
from datetime import datetime

In [ ]:
from memes_text_image import (
    process_images3, process_dataframe, classify_sentiment, 
    determine_object_count,  map_to_category,  categorize_time, detect_objects_and_emotions, 
    extract_text_from_image,classify_category, normalize_upvotes )

In [ ]:

df_no_duplicates =pd.read_csv('/data/notebook_files/Reddit_Project/memes_metadata.csv')
process_images3(df_no_duplicates, "/data/notebook_files/Further_Process/FinalDataset/Dataset", "output_csv")
output101 = pd.read_csv('/data/notebook_files/Reddit_Project/output_csv')

In [ ]:
new_folder = '/data/notebook_files/Further_Process/FinalDataset/Dataset'

model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 
emotion_model = FER()
output101[['Detected Objects', 'Facial Expression']] = output101['Filename'].apply(
    lambda filename: pd.Series(detect_objects_and_emotions(model,emotion_model,os.path.join(new_folder, filename)))
)
output101_with_text = process_dataframe(output101, new_folder)

In [ ]:
output101_with_text['Objects_ind'] = output101_with_text['Detected Objects'].apply(lambda x: 0 if pd.isna(x) or x == '' or x.isspace() else 1)
output101_with_text['object_count'] = output101_with_text['Detected Objects'].apply(determine_object_count)
output101_with_text = pd.get_dummies(output101_with_text, columns=['object_count'], prefix='object_count')
output101_with_text['do_category'] = output101_with_text['Detected Objects'].apply(map_to_category)
one_hot_encoded = pd.get_dummies(output101_with_text['do_category'], prefix='do_category')
output101_with_text = pd.concat([output101_with_text, one_hot_encoded], axis=1)
# Replace missing values in 'Facial Expression' with 'unknown'
output101_with_text['Facial Expression'].fillna('unknown', inplace=True)
output101_with_text = pd.get_dummies(output101_with_text, columns=['Facial Expression'], prefix='Facial_Expression')

In [ ]:
sentiment_analyzer = pipeline('sentiment-analysis')
output101_with_text['sentiment'] = output101_with_text['Extracted_Text'].apply(classify_sentiment)
output101_with_text = pd.get_dummies(output101_with_text, columns=['sentiment'], prefix='meme_text_sentiment')

In [ ]:
output101_with_text['UTC'] = output101_with_text['UTC'].apply(lambda x: datetime.utcfromtimestamp(x))
output101_with_text['Time'] = output101_with_text['UTC'].apply(categorize_time)
output101_with_text = pd.get_dummies(output101_with_text, columns=['Time'], prefix='Time')

In [ ]:
cta_words = ['Comment',  'post', 'View', 'Reply', 'Posted', 'Follow', 'Share',  'Share Save']
output101_with_text['call_to_action_ind'] = output101_with_text['Extracted_Text'].apply(lambda text: 1 if text and any(word.lower() in str(text).lower() for word in cta_words) else 0)
output101_with_text['submission_text_ind'] = output101_with_text['Submission Text'].notnull().astype(int)
output101_with_text['NSFW_ind'] = output101_with_text['NSFW'].astype(int)
output101_with_text['title_word_count'] = output101_with_text['Title'].str.split().apply(len)


In [ ]:
text_classifier = pipeline("sentiment-analysis")
output101_with_text['title_sentiment'] = output101_with_text['Title'].apply(classify_category)
category_one_hot = pd.get_dummies(output101_with_text['title_sentiment'], prefix='title_sentiment')
output101_with_text = pd.concat([output101_with_text, category_one_hot], axis=1)

In [ ]:
num_ventiles = 20
output101_with_text = output101_with_text.groupby('Subreddit').apply(normalize_upvotes)
output101_with_text = list(range(1, num_ventiles + 1))
output101_with_text['Ventile'] = pd.qcut(output101_with_text['Normalized_Upvotes'], q=num_ventiles, labels=ventile_labels, duplicates='drop')

In [ ]:
output101_with_text.to_csv(metadata_with_features.csv)